# Lotka Volterra

In [1]:
using Catlab, DataMigrations, AlgebraicRewriting
using Random, Test, StructEquality
using Luxor

Random.seed!(123)

using Catlab.Graphics.Graphviz: Attributes, Statement, Node
using Catlab.Graphics.Graphviz

import Catlab.CategoricalAlgebra: left, right

function right(s::Symbol)
  if s == :N
    return :E
  elseif s == :S
    return :W
  elseif s == :E
    return :S
  elseif s == :W
    return :N
  end
end
function left(s::Symbol)
  if s == :N
    return :W
  elseif s == :S
    return :E
  elseif s == :E
    return :N
  elseif s == :W
    return :S
  end
end

left (generic function with 7 methods)

Grass = 0 means alive grass, whereas grass > 0 represent a counter of time until the grass is alive.

Sheeps and wolves have position and direction, so we assign each an *edge*. We assume a convention where the location of the something is the edge SOURCE.

Dir is an attribute which can take values :N, :E, :W, and :S.

In [2]:
@present TheoryLV <: SchGraph begin
  (Sheep, Wolf)::Ob
  sheep_loc::Hom(Sheep, V)
  wolf_loc::Hom(Wolf, V)

  (Dir, Eng)::AttrType
  grass_eng::Attr(V, Eng)
  sheep_eng::Attr(Sheep, Eng)
  wolf_eng::Attr(Wolf, Eng)
  sheep_dir::Attr(Sheep, Dir)
  wolf_dir::Attr(Wolf, Dir)
  dir::Attr(E, Dir)
end

@present TheoryLV′ <: TheoryLV begin
  Coord::AttrType
  coord::Attr(V, Coord)
end

to_graphviz(TheoryLV; prog="dot")

@acset_type LV_Generic(TheoryLV) <: HasGraph
const LV = LV_Generic{Symbol,Int}

@acset_type LV′_Generic(TheoryLV′) <: HasGraph
const LV′ = LV′_Generic{Symbol,Int,Tuple{Int,Int}}

F = Migrate(
  Dict(:Sheep => :Wolf, :Wolf => :Sheep),
  Dict([:sheep_loc => :wolf_loc, :wolf_loc => :sheep_loc,
    :sheep_eng => :wolf_eng, :wolf_eng => :sheep_eng, :grass_eng => :grass_eng,
    :sheep_dir => :wolf_dir, :wolf_dir => :sheep_dir,]), LV)
F2 = Migrate(
  Dict(x => x for x in Symbol.(TheoryLV.generators[:Ob])),
  Dict(x => x for x in Symbol.(TheoryLV.generators[:Hom])), LV′; delta=false)

Migrate(Dict(:Wolf => :Wolf, :V => :V, :Sheep => :Sheep, :E => :E), Dict(:wolf_loc => :wolf_loc, :src => :src, :sheep_loc => :sheep_loc, :tgt => :tgt), Main.var"##232".LV′_Generic{Symbol, Int64, Tuple{Int64, Int64}}, Main.var"##232".LV′_Generic{Symbol, Int64, Tuple{Int64, Int64}}, false)

Create an n × n grid with periodic boundary conditions. Edges in each cardinal
direction originate at every point

(i,j+1) -> (i+1,j+1) -> ...
  ↑          ↑
(i,j)   -> (i+1,j)   -> ...

In [3]:
function create_grid(n::Int)
  lv = LV′()
  coords = Dict()
  for i in 0:n-1  # Initialize grass 50% green, 50% uniformly between 0-30
    for j in 0:n-1
      coords[i=>j] = add_part!(lv, :V; grass_eng=max(0, rand(-30:30)), coord=(i, j))
    end
  end
  for i in 0:n-1
    for j in 0:n-1
      add_part!(lv, :E; src=coords[i=>j], tgt=coords[mod(i + 1, n)=>j], dir=:E)
      add_part!(lv, :E; src=coords[i=>j], tgt=coords[mod(i - 1, n)=>j], dir=:W)
      add_part!(lv, :E; src=coords[i=>j], tgt=coords[i=>mod(j + 1, n)], dir=:N)
      add_part!(lv, :E; src=coords[i=>j], tgt=coords[i=>mod(j - 1, n)], dir=:S)
    end
  end
  return lv
end

g = create_grid(2)

Main.var"##232".LV′_Generic{Symbol, Int64, Tuple{Int64, Int64}} {V:4, E:16, Sheep:0, Wolf:0, Dir:0, Eng:0, Coord:0}
┌───┬───────────┬────────┐
│ V │ grass_eng │  coord │
├───┼───────────┼────────┤
│ 1 │         1 │ (0, 0) │
│ 2 │         5 │ (0, 1) │
│ 3 │        24 │ (1, 0) │
│ 4 │         0 │ (1, 1) │
└───┴───────────┴────────┘
┌────┬─────┬─────┬─────┐
│  E │ src │ tgt │ dir │
├────┼─────┼─────┼─────┤
│  1 │   1 │   3 │   E │
│  2 │   1 │   3 │   W │
│  3 │   1 │   2 │   N │
│  4 │   1 │   2 │   S │
│  5 │   2 │   4 │   E │
│  6 │   2 │   4 │   W │
│  7 │   2 │   1 │   N │
│  8 │   2 │   1 │   S │
│  9 │   3 │   1 │   E │
│ 10 │   3 │   1 │   W │
│ 11 │   3 │   4 │   N │
│ 12 │   3 │   4 │   S │
│ 13 │   4 │   2 │   E │
│ 14 │   4 │   2 │   W │
│ 15 │   4 │   3 │   N │
│ 16 │   4 │   3 │   S │
└────┴─────┴─────┴─────┘

`n` is the length of the grid.
`sheep` and `wolves` are the fraction of spaces that are
populated with that animal

In [4]:
function initialize(n::Int, sheep::Float64, wolves::Float64)::LV′
  grid = create_grid(n)
  args = [(sheep, :Sheep, :sheep_loc, :sheep_eng, :sheep_dir),
    (wolves, :Wolf, :wolf_loc, :wolf_eng, :wolf_dir)]
  for (n_, name, loc, eng, d) in args
    for _ in 1:round(Int, n_ * n^2)
      dic = Dict([eng => 5, loc => rand(vertices(grid)),
        d => rand([:N, :E, :S, :W])])
      add_part!(grid, name; dic...)
    end
  end
  return grid
end


supscript_d = Dict([
  '1' => '¹', '2' => '²', '3' => '³', '4' => '⁴', '5' => '⁵', '6' => '⁶', '7' => '⁷', '8' => '⁸',
  '9' => '⁹', '0' => '⁰', 'x' => 'ˣ', 'y' => 'ʸ', 'z' => 'ᶻ', 'a' => 'ᵃ', 'b' => 'ᵇ', 'c' => 'ᶜ',
  'd' => 'ᵈ'])
supscript(x::String) = join([get(supscript_d, c, c) for c in x])

supscript (generic function with 1 method)

Visualize a LV

In [5]:
function view_LV(p::ACSetTransformation, pth=tempname(); name="G", title="")
  if nparts(dom(p), :Wolf) == 1
    star = :Wolf => p[:Wolf](1)
  elseif nparts(dom(p), :Sheep) == 1
    star = :Sheep => p[:Sheep](1)
  elseif nparts(dom(p), :V) == 1
    star = :V => p[:V](1)
  else
    star = nothing
  end
  view_LV(codom(p), pth; name=name, title=title, star=star)
end
function view_LV(p::LV′, pth=tempname(); name="G", title="", star=nothing)
  pstr = ["$(i),$(j)!" for (i, j) in p[:coord]]
  stmts = Statement[]
  for s in 1:nv(p)
    st = (star == (:V => s)) ? "*" : ""
    gv = p[s, :grass_eng]
    col = gv == 0 ? "lightgreen" : "tan"
    push!(stmts, Node("v$s", Attributes(
      :label => gv == 0 ? "" : string(gv) * st,
      :shape => "circle",
      :color => col, :pos => pstr[s])))
  end
  d = Dict([:E => (1, 0), :N => (0, 1), :S => (0, -1), :W => (-1, 0),])

  args = [(:true, :Wolf, :wolf_loc, :wolf_eng, :wolf_dir),
    (false, :Sheep, :sheep_loc, :sheep_eng, :sheep_dir)]

  for (is_wolf, prt, loc, eng, dr) in args
    for w in parts(p, prt)
      st = (star == ((is_wolf ? :Wolf : :Sheep) => w)) ? "*" : ""
      e = only(incident(p, p[w, loc], :src) ∩ incident(p, p[w, dr], :dir))
      s = src(p, e)
      dx, dy = d[p[e, :dir]]
      (sx, sy) = p[s, :coord]

      L, R = 0.25, 0.1
      wx = sx + L * dx + R * rand()
      wy = sy + L * dy + R * rand()
      ID = "$(is_wolf ? :w : :s)$w"
      append!(stmts, [Node(ID, Attributes(
        :label => "$w" * supscript("$(p[w,eng])") * st,
        :shape => "square", :width => "0.3px", :height => "0.3px", :fixedsize => "true",
        :pos => "$(wx),$(wy)!", :color => is_wolf ? "red" : "lightblue"))])
    end
  end

  g = Graphviz.Digraph(name, Statement[stmts...]; prog="neato",
    graph_attrs=Attributes(:label => title, :labelloc => "t"),
    node_attrs=Attributes(:shape => "plain", :style => "filled"))
  open(pth, "w") do io
    show(io, "image/svg+xml", g)
  end
end

i1 = initialize(2, 0.5, 0.5)

Main.var"##232".LV′_Generic{Symbol, Int64, Tuple{Int64, Int64}} {V:4, E:16, Sheep:2, Wolf:2, Dir:0, Eng:0, Coord:0}
┌───┬───────────┬────────┐
│ V │ grass_eng │  coord │
├───┼───────────┼────────┤
│ 1 │         2 │ (0, 0) │
│ 2 │         0 │ (0, 1) │
│ 3 │         0 │ (1, 0) │
│ 4 │        26 │ (1, 1) │
└───┴───────────┴────────┘
┌────┬─────┬─────┬─────┐
│  E │ src │ tgt │ dir │
├────┼─────┼─────┼─────┤
│  1 │   1 │   3 │   E │
│  2 │   1 │   3 │   W │
│  3 │   1 │   2 │   N │
│  4 │   1 │   2 │   S │
│  5 │   2 │   4 │   E │
│  6 │   2 │   4 │   W │
│  7 │   2 │   1 │   N │
│  8 │   2 │   1 │   S │
│  9 │   3 │   1 │   E │
│ 10 │   3 │   1 │   W │
│ 11 │   3 │   4 │   N │
│ 12 │   3 │   4 │   S │
│ 13 │   4 │   2 │   E │
│ 14 │   4 │   2 │   W │
│ 15 │   4 │   3 │   N │
│ 16 │   4 │   3 │   S │
└────┴─────┴─────┴─────┘
┌───────┬───────────┬───────────┬───────────┐
│ Sheep │ sheep_loc │ sheep_eng │ sheep_dir │
├───────┼───────────┼───────────┼───────────┤
│     1 │         3 │         5 │         E │
│     2 │         3 │         5 │         W │
└───────┴───────────┴───────────┴───────────┘
┌──────┬──────────┬──────────┬──────────┐
│ Wolf │ wolf_loc │ wolf_eng │ wolf_dir │
├──────┼──────────┼──────────┼──────────┤
│    1 │        1 │        5 │        E │
│    2 │        2 │        5 │        N │
└──────┴──────────┴──────────┴──────────┘

# Rules

In [6]:
yLV = yoneda_cache(LV; clear=true);
yLV = yoneda_cache(LV; clear=false);

Empty agent type

In [7]:
I = LV()

Main.var"##232".LV_Generic{Symbol, Int64} {V:0, E:0, Sheep:0, Wolf:0, Dir:0, Eng:0}

Generic sheep agent

In [8]:
S = @acset_colim yLV begin
  s::Sheep
end

Main.var"##232".LV_Generic{Symbol, Int64} {V:1, E:0, Sheep:1, Wolf:0, Dir:1, Eng:2}
┌───┬────────────┐
│ V │  grass_eng │
├───┼────────────┤
│ 1 │ AttrVar(2) │
└───┴────────────┘
┌───────┬───────────┬────────────┬────────────┐
│ Sheep │ sheep_loc │  sheep_eng │  sheep_dir │
├───────┼───────────┼────────────┼────────────┤
│     1 │         1 │ AttrVar(1) │ AttrVar(1) │
└───────┴───────────┴────────────┴────────────┘

Generic wolf agent

In [9]:
W = F(S)

Main.var"##232".LV_Generic{Symbol, Int64} {V:1, E:0, Sheep:0, Wolf:1, Dir:1, Eng:2}
┌───┬────────────┐
│ V │  grass_eng │
├───┼────────────┤
│ 1 │ AttrVar(2) │
└───┴────────────┘
┌──────┬──────────┬────────────┬────────────┐
│ Wolf │ wolf_loc │   wolf_eng │   wolf_dir │
├──────┼──────────┼────────────┼────────────┤
│    1 │        1 │ AttrVar(1) │ AttrVar(1) │
└──────┴──────────┴────────────┴────────────┘

Generic grass agent

In [10]:
G = @acset_colim yLV begin
  v::V
end

N = Names(Dict("W" => W, "S" => S, "G" => G, "" => I));

## Rotating

In [11]:
rl = Rule(id(S), id(S); expr=(Dir=[xs -> left(only(xs))],))
rr = Rule(id(S), id(S); expr=(Dir=[xs -> right(only(xs))],))

sheep_rotate_l = tryrule(RuleApp(:turn_left, rl, S))
sheep_rotate_r = tryrule(RuleApp(:turn_right, rr, S))

Schedule(WiringDiagram{AlgebraicRewriting.Schedules.Theories.ThTracedMonoidalWithBidiagonals.Meta.T}([Main.var"##232".LV_Generic{Symbol, Int64} {V:1, E:0, Sheep:1, Wolf:0, Dir:1, Eng:2}], [Main.var"##232".LV_Generic{Symbol, Int64} {V:1, E:0, Sheep:1, Wolf:0, Dir:1, Eng:2}], 
[ -2 => {inputs},
  -1 => {outputs},
  1 => Box("turn_right", [Main.var"##232".LV_Generic{Symbol, Int64}:
  V = 1:1
  E = 1:0
  Sheep = 1:1
  Wolf = 1:0
  Dir = 1:1
  Eng = 1:2
  src : E → V = Int64[]
  tgt : E → V = Int64[]
  sheep_loc : Sheep → V = [1]
  wolf_loc : Wolf → V = Int64[]
  grass_eng : V → Eng = ACSets.ColumnImplementations.AttrVar[ACSets.ColumnImplementations.AttrVar(2)]
  sheep_eng : Sheep → Eng = ACSets.ColumnImplementations.AttrVar[ACSets.ColumnImplementations.AttrVar(1)]
  wolf_eng : Wolf → Eng = Int64[]
  sheep_dir : Sheep → Dir = ACSets.ColumnImplementations.AttrVar[ACSets.ColumnImplementations.AttrVar(1)]
  wolf_dir : Wolf → Dir = Symbol[]
  dir : E → Dir = Symbol[]], [Main.var"##232".LV_Gener

We can imagine executing these rules in sequence or in parallel

In [12]:
seq_sched = (sheep_rotate_l ⋅ sheep_rotate_r)
par_sched = (sheep_rotate_l ⊗ sheep_rotate_r)

begin
  ex = @acset_colim yLV begin
    e::E
    s::Sheep
    sheep_loc(s) == src(e)
    sheep_dir(s) == :N
  end
  expected = copy(ex)
  expected[:sheep_dir] = :W
  @test is_isomorphic(rewrite(rl, ex), expected)
end

Test Passed

## Moving forward

In [13]:
s_fwd_l = @acset_colim yLV begin
  e::E
  s::Sheep
  sheep_loc(s) == src(e)
end
s_fwd_i = @acset_colim yLV begin
  e::E
end
s_fwd_r = @acset_colim yLV begin
  e::E
  s::Sheep
  sheep_loc(s) == tgt(e)
end
s_n = @acset_colim yLV begin
  e::E
  s::Sheep
  sheep_loc(s) == src(e)
  sheep_eng(s) == 0
end

sheep_fwd_rule = Rule(
  homomorphism(s_fwd_i, s_fwd_l; monic=true),
  homomorphism(s_fwd_i, s_fwd_r; monic=true),
  ac=[AppCond(homomorphism(s_fwd_l, s_n), false)],
  expr=(Eng=Dict(3 => vs -> vs[3] - 1), Dir=Dict(2 => vs -> vs[2]))
)

sheep_fwd = tryrule(RuleApp(:move_fwd, sheep_fwd_rule,
  homomorphism(S, s_fwd_l), homomorphism(S, s_fwd_r)))


sheep_fwd_rule.L |> codom

begin # test
  ex = @acset_colim yLV begin
    (e1, e2)::E
    s::Sheep
    sheep_loc(s) == tgt(e1)
    tgt(e1) == src(e2)
    sheep_dir(s) == :N
    sheep_eng(s) == 10
  end
  expected = @acset_colim yLV begin
    (e1, e2)::E
    s::Sheep
    sheep_loc(s) == tgt(e2)
    tgt(e1) == src(e2)
    sheep_dir(s) == :N
    sheep_eng(s) == 9
  end
  @test is_isomorphic(expected, rewrite(sheep_fwd_rule, ex))
end

Test Passed

Eat grass + 4eng
Grass is at 0 - meaning it's ready to be eaten

In [14]:
s_eat_pac = @acset_colim yLV begin
  s::Sheep
  grass_eng(sheep_loc(s)) == 0
end

se_rule = Rule(id(S), id(S); expr=(Eng=[vs -> vs[1] + 4, vs -> 30],),
  ac=[AppCond(homomorphism(S, s_eat_pac))])
sheep_eat = tryrule(RuleApp(:Sheep_eat, se_rule, S))

begin # test
  ex = @acset_colim yLV begin
    s::Sheep
    e::E
    sheep_loc(s) == tgt(e)
    sheep_eng(s) == 3
    grass_eng(tgt(e)) == 0
    grass_eng(src(e)) == 10
  end
  expected = @acset_colim yLV begin
    s::Sheep
    e::E
    sheep_loc(s) == tgt(e)
    sheep_eng(s) == 7
    grass_eng(tgt(e)) == 30
    grass_eng(src(e)) == 10
  end

  @test is_isomorphic(expected, rewrite(se_rule, ex))
end

Test Passed

Eat sheep + 20 eng

In [15]:
w_eat_l = @acset_colim yLV begin
  s::Sheep
  w::Wolf
  sheep_loc(s) == wolf_loc(w)
end

we_rule = Rule(homomorphism(W, w_eat_l), id(W); expr=(Eng=[vs -> vs[3] + 20, vs -> vs[1]],))
wolf_eat = tryrule(RuleApp(:Wolf_eat, we_rule, W))

Schedule(WiringDiagram{AlgebraicRewriting.Schedules.Theories.ThTracedMonoidalWithBidiagonals.Meta.T}([Main.var"##232".LV_Generic{Symbol, Int64} {V:1, E:0, Sheep:0, Wolf:1, Dir:1, Eng:2}], [Main.var"##232".LV_Generic{Symbol, Int64} {V:1, E:0, Sheep:0, Wolf:1, Dir:1, Eng:2}], 
[ -2 => {inputs},
  -1 => {outputs},
  1 => Box("Wolf_eat", [Main.var"##232".LV_Generic{Symbol, Int64}:
  V = 1:1
  E = 1:0
  Sheep = 1:0
  Wolf = 1:1
  Dir = 1:1
  Eng = 1:2
  src : E → V = Int64[]
  tgt : E → V = Int64[]
  sheep_loc : Sheep → V = Int64[]
  wolf_loc : Wolf → V = [1]
  grass_eng : V → Eng = ACSets.ColumnImplementations.AttrVar[ACSets.ColumnImplementations.AttrVar(2)]
  sheep_eng : Sheep → Eng = Int64[]
  wolf_eng : Wolf → Eng = ACSets.ColumnImplementations.AttrVar[ACSets.ColumnImplementations.AttrVar(1)]
  sheep_dir : Sheep → Dir = Symbol[]
  wolf_dir : Wolf → Dir = ACSets.ColumnImplementations.AttrVar[ACSets.ColumnImplementations.AttrVar(1)]
  dir : E → Dir = Symbol[]], [Main.var"##232".LV_Generic

### A test

In [16]:
ex = @acset LV begin
  Sheep = 1
  Wolf = 1
  V = 3
  E = 2
  src = [1, 2]
  tgt = [2, 3]
  sheep_loc = 2
  sheep_eng = [3]
  grass_eng = [9, 10, 11]
  dir = fill(:N, 2)
  sheep_dir = [:N]
  wolf_loc = [2]
  wolf_eng = [16]
  wolf_dir = [:S]
end
expected = @acset LV begin
  Wolf = 1
  V = 3
  E = 2
  src = [1, 2]
  tgt = [2, 3]
  grass_eng = [9, 10, 11]
  dir = fill(:N, 2)
  sheep_dir = [:N]
  wolf_loc = [2]
  wolf_eng = [36]
  wolf_dir = [:S]
end
@test is_isomorphic(rewrite(we_rule, ex), expected)

Test Passed

Die if 0 eng

In [17]:
s_die_l = @acset_colim yLV begin
  s::Sheep
  sheep_eng(s) == 0
end

sheep_die_rule = Rule(homomorphism(G, s_die_l), id(G))
sheep_starve = (RuleApp(:starve, sheep_die_rule,
  homomorphism(S, s_die_l), create(G))
                ⋅
                (id([I]) ⊗ Weaken(create(S))) ⋅ merge_wires(I))

begin # test
  ex = s_die_l ⊕ W
  expected = G ⊕ W
  @test is_isomorphic(rewrite(sheep_die_rule, ex), expected)
end

Test Passed

Reproduction

In [18]:
s_reprod_r = @acset_colim yLV begin
  (x, y)::Sheep
  sheep_loc(x) == sheep_loc(y)
end

sheep_reprod_rule = Rule(
  homomorphism(G, S),
  homomorphism(G, s_reprod_r);
  expr=(Dir=[vs -> vs[1], vs -> vs[1]], Eng=[vs -> vs[2],
    fill(vs -> round(Int, vs[1] / 2, RoundUp), 2)...],)
)

sheep_reprod = RuleApp(:reproduce, sheep_reprod_rule,
  id(S), homomorphism(S, s_reprod_r)) |> tryrule

begin # test
  ex = @acset_colim yLV begin
    s::Sheep
    w::Wolf
    sheep_eng(s) == 10
  end
  expected = @acset_colim yLV begin
    (s1, s2)::Sheep
    w::Wolf
    sheep_loc(s1) == sheep_loc(s2)
    sheep_eng(s1) == 5
    sheep_eng(s2) == 5
  end
  @test is_isomorphic(rewrite(sheep_reprod_rule, ex), expected)
end

Test Passed

Grass increment

In [19]:
g_inc_n = deepcopy(G)
set_subpart!(g_inc_n, 1, :grass_eng, 0)
rem_part!(g_inc_n, :Eng, 1)

g_inc_rule = Rule(id(G), id(G);
  ac=[AppCond(homomorphism(G, g_inc_n), false)],
  expr=(Eng=[vs -> only(vs) - 1],))
g_inc = RuleApp(:GrassIncrements, g_inc_rule, G) |> tryrule


ex = @acset LV begin
  Sheep = 1
  V = 3
  E = 2
  src = [1, 2]
  tgt = [2, 3]
  sheep_loc = 2
  sheep_eng = [3]
  grass_eng = [1, 10, 2]
  dir = fill(:N, 2)
  sheep_dir = [:N]
end
expected = @acset LV begin
  Sheep = 1
  V = 3
  E = 2
  src = [1, 2]
  tgt = [2, 3]
  sheep_loc = 2
  sheep_eng = [3]
  grass_eng = [0, 10, 2]
  dir = fill(:N, 2)
  sheep_dir = [:N]
end
@test is_isomorphic(rewrite(g_inc_rule, ex), expected)

Test Passed

Scheduling Rules

In [20]:
general = mk_sched((;), (init=:S,), N, (
    turn=const_cond([1.0, 2.0, 1.0], S; name=:turn),
    maybe=const_cond([0.1, 0.9], S; name=:reprod),
    lft=sheep_rotate_l,
    rght=sheep_rotate_r,
    fwd=sheep_fwd,
    repro=sheep_reprod,
    starve=sheep_starve),
  quote
    out_l, out_str, out_r = turn(init)
    moved = fwd([lft(out_l), out_str, rght(out_r)])
    out_repro, out_no_repro = maybe(moved)
    return starve([repro(out_repro), out_no_repro])
  end)

sheep = sheep_eat ⋅ general;   # once per sheep
wolf = wolf_eat ⋅ F(general);  # once per wolf

Do all sheep, then all wolves, then all daily operations

In [21]:
cycle = (agent(sheep; n=:sheep, ret=I)
         ⋅
         agent(wolf; n=:wolves, ret=I)
         ⋅
         agent(g_inc; n=:grass))

overall = while_schedule(cycle, curr -> nparts(curr, :Wolf) >= 0) |> F2  # wrap in a while loop

X = initialize(3, 0.25, 0.25);
res, = apply_schedule(overall, X; steps=50);